# Image rotation with CNN

this is a search and development project to do on the weekends.


my main resourse is this blog post: https://d4nst.github.io/2017/01/12/image-orientation/



In [1]:
targetUrl = 'datasets/streetview'
gmap_images_url = "http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images"


## Rotate images
#### Get images

In [ ]:
import math
# generate filenames
# example 000005_3.jpg
# the postfix can be a number between 0 and 5
# the first part is a simple xml padded to 6 char: '{0:0>6}'.format('132')
fileNames = []
max = 10
for i in range(1, math.floor(max/6)+1):
    baseName = '{0:0>6}'.format(i);
    for postfix in range(6):
        fileNames.append(baseName + "_" + str(postfix) + ".jpg");


In [ ]:
fileNames[0:10]

In [ ]:
import urllib.request


for filename in fileNames:
    print('downloading {0}\n'.format(filename))
    urllib.request.urlretrieve(gmap_images_url + '/' + filename, targetUrl + '/' + filename)


### rotate image in 3 orientation, copy and add some label

labels: https://www.impulseadventure.com/photo/exif-orientation.html

- 1 top, 
- 8 left side, 
- 3 bottom, 
- 6 right side

prepend this label to the filename separted by a "#", eg. `1#000001_5.jpg`



In [ ]:
import os
from PIL import Image

ORIENTATION_LABELS = { 'top': 1, 'left':  8, 'bottom': 3, 'right': 6 }


def rotateImage(orientation, fileName, labeledFileName, subdir):
    anglesForOrientation = { ORIENTATION_LABELS['top']: 0, 
                             ORIENTATION_LABELS['left']: 90, 
                             ORIENTATION_LABELS['bottom']: 180, 
                            ORIENTATION_LABELS['right']: 270 }
    
    if not orientation in anglesForOrientation.keys():
        raise Exception('Unknown orienation: ' + orientation)
    
    angle = anglesForOrientation[orientation]
    
    # crop the image to have equal sizes
    targetPath = os.path.join(subdir, labeledFileName)  
    img = Image.open(os.path.join(subdir, fileName))
    size = 0, 0, 1024, 1024
    img.crop(size).rotate(angle).save(targetPath)
                                  
#rotateImage(ORIENTATION_LABELS['bottom'], 1, 2, 3)

In [ ]:
# first iterate them and add the appropriate label
import os


def isImage(file):
    return '.jpg' in file

def isProcessed(file):
    return '#' in file

def addLabelToFile(label, file):
    return '{label}#{file}'.format(label = label, file = file)


def rotateImages(fileName, subdir):
    for orientation in ORIENTATION_LABELS.values():
        labeledFileName = addLabelToFile(orientation, fileName)
        rotateImage(orientation, fileName, labeledFileName, subdir)




for subdir, dirs, files in os.walk(targetUrl):
    for file in files:
        if (not isImage(file)):
            continue
        if (isProcessed(file)):
            continue
        print ('Processing: {0}'.format(os.path.join(subdir, file)))
        
        
        # create the rotations
        rotateImages(file, subdir)
        
        # delete originals
        os.remove(os.path.join(subdir, file))
        

# convert images into classified matrices before train / test split

Keras has a fit_generator where the first param is a generator to add datas on the fly (by batches)

https://keras.io/models/sequential/


In [27]:
import os
import numpy as np
from PIL import Image
import re


class trainDataGenerator:
    def __init__(self, image_src_path, batch_size=64):
        self.image_src_path = image_src_path
        self.batch_size = batch_size
        self.input_shape = (1024, 1024, 3)
        self.files = []
        self._getFilenames()
        print("number of images: {}".format(len(self.files)))
        self.curr_image_index = -1
        self.batch_X = False
        self.batch_y = False
        self.batch_index = -1

        
    def __iter__(self):
        return self

    def __next__(self):
        # create array to hold the images
        self.batch_X = np.zeros((self.batch_size,) + self.input_shape, dtype='float32')
        print("batch_X shape: {}".format(self.batch_X.shape))
        # create array to hold the labels
        self.batch_y = np.zeros(self.batch_size, dtype='int')
        print(self.curr_image_index)
        first_image_in_batch = self.curr_image_index + 1
        last_image_in_batch = first_image_in_batch + self.batch_size
        self.batch_index = -1

        self._processFiles(first_image_in_batch,  last_image_in_batch)    
        return self.batch_X, self.batch_y
    
    def _processFiles(self, first_image_in_batch, last_image_in_batch):
        print('first: {}, second: {}'.format(first_image_in_batch, last_image_in_batch))
        for i in range(first_image_in_batch, last_image_in_batch):
            try:
                file_name = self.files[i]
                self.curr_image_index = i
                img = Image.open(file_name)
                arr = np.array(img)
                # image as numpy array
                shape = arr.shape
                print('{image} has shape: {shape}'.format(image=file_name, shape=shape))
                # batch_X index is always between 0 and batch size, but index starts at the first image in batch.
                # when the batch is 4, first will be 0, then 4, then 8 

                self.batch_index += 1
                print(ebatch_index)
                self.batch_X[batch_index] = arr
                
                # get the label
                found = re.search(r'([1,8,3,6])#.*?', file_name)
                print('before')


                label = found.group(1)
                print('label: {label}'.format(label=label))
                self.batch_y[i] = label
            
            except FileNotFoundError:
                print('cannot open: ' + file_name)
            except IndexError:
                print('stop iteration')
                raise StopIteration()
    
    def _getFilenames(self):
        for _, _, files in os.walk(self.image_src_path):
            for file in files:
                 if '.jpg' in file:
                    self.files.append(self.image_src_path + '/' + file)  
                            
                

In [28]:
for x, y in trainDataGenerator(targetUrl, 4):
    print('1 run')
    print(x.shape, y)

number of images: 20
batch_X shape: (4, 1024, 1024, 3)
-1
first: 0, second: 4
datasets/streetview/1#000001_1.jpg has shape: (1024, 1024, 3)


UnboundLocalError: local variable 'batch_index' referenced before assignment